## Junta tablas

In [29]:
import os
import pandas as pd
import re

In [30]:
def read_and_join_csvs_by_date(fecha, folder_path='1_datos_raw'):
    """
    Función que toma una fecha en formato 'yyyymmdd' y busca los archivos CSV 
    en el folder_path que comiencen con esa fecha, añadiendo la fecha como columna
    y realizando un outer join entre ellos.

    Args:
    -(fecha) (str): Fecha en formato 'yyyymmdd'.
    - folder_path (str): Directorio donde se encuentran los archivos CSV. Por defecto, es el directorio actual.

    Returns:
    - pd.DataFrame: DataFrame resultante del outer join entre los CSV encontrados con la misma fecha,
                    incluyendo una columna "fecha".
    """
    # Listar todos los archivos en la carpeta
    files = os.listdir(folder_path)
    
    # Filtrar los archivos que comienzan con la fecha proporcionada
    csv_files = [f for f in files if f.startswith(fecha) and f.endswith('.csv')]

    # Verificar que haya más de un archivo para realizar el join
    if len(csv_files) > 1:
        # Leer los archivos con la misma fecha y añadir la columna "fecha"
        dfs = []
        for f in csv_files:
            df = pd.read_csv(os.path.join(folder_path, f))
            df['fecha'] =(fecha)  # Añadir la columna con la fecha
            dfs.append(df)

        # Hacer un join de todos los DataFrames de esa fecha usando outer join
        merged_df = dfs[0]
        for df in dfs[1:]:
            merged_df = merged_df.merge(df, how='outer')  # Outer join para añadir registros no coincidentes
        
        return merged_df
    else:
        print(f"No se encontraron suficientes archivos con la fecha {fecha} para hacer un join.")
        return None

# Uso de la función
fecha = '20241114'  # Fecha en formato 'yyyymmdd'
df = read_and_join_csvs_by_date(fecha)
del df['Unnamed: 0']
df

,id,nombre,precio,estado,reservado,url,fecha
0,1068541305,iphone 16 128gb,"800,00",new,False,https://es.wallapop.com/item/iphone-16-128gb-1...,20241114
1,1069277594,iphone 14 pro morado,"700,00",new,False,https://es.wallapop.com/item/iphone-14-pro-mor...,20241114
2,1069385453,iphone 16,"850,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-16-1069385453,20241114
3,1069408278,iphone 14 pro,"800,00",good,False,https://es.wallapop.com/item/iphone-14-pro-106...,20241114
4,1069426273,iphone 16 pro max,"1020,00",good,False,https://es.wallapop.com/item/iphone-16-pro-max...,20241114
...,...,...,...,...,...,...,...
1453,1064280078,iphone 14 plus 128gb,"589,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-14-plus-12...,20241114
1454,1063248685,iphone 14 pro,"675,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-14-pro-106...,20241114
1455,1063827942,iphone 14 pro max,"850,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-14-pro-max...,20241114
1456,1064909050,iphone 14 pro max 128gb nuevo,"799,00",as_good_as_new,False,https://es.wallapop.com/item/iphone-14-pro-max...,20241114


In [31]:
# Creo una columna en la que pongo que iphone es (14, 15 o 16..) y que tipo. 
# Gen
# Model
for index in range(len(df)):
    df['gen'] = 'vacio'
    df['modelo'] = 'modelo'


In [32]:
def asignar_gen(nombre):
    if '14' in nombre:
        return '14'
    elif '15' in nombre:
        return '15'
    elif '16' in nombre:
        return '16'
    return "n/a"

# Función para asignar el valor de la columna "modelo"
def asignar_modelo(nombre):
    if 'pro max' in nombre.lower():
        return 'pro max'
    elif 'pro' in nombre.lower():
        return 'pro'
    elif 'plus' in nombre.lower():
        return 'plus'
    else:
        return 'basic'

def asignar_memoria(nombre):
    if '128' in nombre.lower():
        return '128'
    elif '256' in nombre.lower():
        return '256'
    elif '512' in nombre.lower():
        return '512'
    elif '1tb' in nombre.lower():
        return '1tb'
    else:
        return 'n/a'

# Función para asignar el valor de la columna "bateria"
def asignar_bateria(nombre):
    # Buscar números entre 80 y 100 en el nombre
    bateria = re.findall(r'\b([8-9][0-9]|100)\b', nombre)
    if bateria:
        return int(bateria[0])  # Retorna el primer número que encuentra
    return "n/a"

# Aplicar las funciones al DataFrame
df['gen'] = df['nombre'].apply(asignar_gen)
df['mod'] = df['nombre'].apply(asignar_modelo)
df['memoria'] = df['nombre'].apply(asignar_memoria)
df['bateria'] = df['nombre'].apply(asignar_bateria)
orden_columnas = ['id','fecha', 'gen', 'mod', 'memoria', 'bateria','precio', 'estado', 'nombre',  'reservado', 'url']
df = df[orden_columnas]
df

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url
0,1068541305,20241114,16,basic,128,n/a,"800,00",new,iphone 16 128gb,False,https://es.wallapop.com/item/iphone-16-128gb-1...
1,1069277594,20241114,14,pro,n/a,n/a,"700,00",new,iphone 14 pro morado,False,https://es.wallapop.com/item/iphone-14-pro-mor...
2,1069385453,20241114,16,basic,n/a,n/a,"850,00",as_good_as_new,iphone 16,False,https://es.wallapop.com/item/iphone-16-1069385453
3,1069408278,20241114,14,pro,n/a,n/a,"800,00",good,iphone 14 pro,False,https://es.wallapop.com/item/iphone-14-pro-106...
4,1069426273,20241114,16,pro max,n/a,n/a,"1020,00",good,iphone 16 pro max,False,https://es.wallapop.com/item/iphone-16-pro-max...
...,...,...,...,...,...,...,...,...,...,...,...
1453,1064280078,20241114,14,plus,128,n/a,"589,00",as_good_as_new,iphone 14 plus 128gb,False,https://es.wallapop.com/item/iphone-14-plus-12...
1454,1063248685,20241114,14,pro,n/a,n/a,"675,00",as_good_as_new,iphone 14 pro,False,https://es.wallapop.com/item/iphone-14-pro-106...
1455,1063827942,20241114,14,pro max,n/a,n/a,"850,00",as_good_as_new,iphone 14 pro max,False,https://es.wallapop.com/item/iphone-14-pro-max...
1456,1064909050,20241114,14,pro max,128,n/a,"799,00",as_good_as_new,iphone 14 pro max 128gb nuevo,False,https://es.wallapop.com/item/iphone-14-pro-max...


In [33]:
carpeta = "2_datos_por_fecha"
nombre_archivo_pkl = carpeta + '/' + fecha + '.csv'
df.to_csv(nombre_archivo_pkl)
print(nombre_archivo_pkl)

2_datos_por_fecha/20241114.csv
